#GoogleAds Segmentology
GoogleAds funnel analysis using Census data.


#License

Copyright 2020 Google LLC,

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

  https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.



#Disclaimer
This is not an officially supported Google product. It is a reference implementation. There is absolutely NO WARRANTY provided for using this code. The code is Apache Licensed and CAN BE fully modified, white labeled, and disassembled by your team.

This code generated (see starthinker/scripts for possible source):
  - **Command**: "python starthinker_ui/manage.py colab"
  - **Command**: "python starthinker/tools/colab.py [JSON RECIPE]"



#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Set Configuration

This code is required to initialize the project. Fill in required fields and press play.

1. If the recipe uses a Google Cloud Project:
  - Set the configuration **project** value to the project identifier from [these instructions](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md).

1. If the recipe has **auth** set to **user**:
  - If you have user credentials:
    - Set the configuration **user** value to your user credentials JSON.
  - If you DO NOT have user credentials:
    - Set the configuration **client** value to [downloaded client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md).

1. If the recipe has **auth** set to **service**:
  - Set the configuration **service** value to [downloaded service credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_service.md).



In [ ]:
from starthinker.util.configuration import Configuration


CONFIG = Configuration(
  project="",
  client={},
  service={},
  user="/content/user.json",
  verbose=True
)



#3. Enter GoogleAds Segmentology Recipe Parameters
 1. Wait for **BigQuery->->->Census_Join** to be created.
 1. Join the [StarThinker Assets Group](https://groups.google.com/d/forum/starthinker-assets) to access the following assets
 1. Copy [GoogleAds Segmentology Sample](https://datastudio.google.com/c/u/0/reporting/3673497b-f36f-4448-8fb9-3e05ea51842f/). Leave the Data Source as is, you will change it in the next step.
 1. Click Edit Connection, and change to **BigQuery->->->Census_Join**.
 1. Or give these instructions to the client.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'auth_read':'user',  # Credentials used for reading data.
  'customer_id':'',  # Google Ads customer.
  'developer_token':'',  # Google Ads developer token.
  'login_id':'',  # Google Ads login.
  'auth_write':'service',  # Authorization used for writing data.
  'recipe_slug':'',  # Name of Google BigQuery dataset to create.
}

print("Parameters Set To: %s" % FIELDS)


#4. Execute GoogleAds Segmentology
This does NOT need to be modified unless you are changing the recipe, click play.


In [ ]:
from starthinker.util.configuration import execute
from starthinker.util.recipe import json_set_fields

TASKS = [
  {
    'dataset':{
      'description':'Create a dataset for bigquery tables.',
      'hour':[
        4
      ],
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing data.'}},
      'dataset':{'field':{'name':'recipe_slug','kind':'string','suffix':'_Segmentology','description':'Place where tables will be created in BigQuery.'}}
    }
  },
  {
    'bigquery':{
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing function.'}},
      'function':'Pearson Significance Test',
      'to':{
        'dataset':{'field':{'name':'recipe_slug','kind':'string','suffix':'_Segmentology','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}}
      }
    }
  },
  {
    'google_api':{
      'auth':{'field':{'name':'auth_read','kind':'authentication','order':0,'default':'user','description':'Credentials used for reading data.'}},
      'api':'googleads',
      'version':'v8',
      'function':'customers.googleAds.search',
      'kwargs':{
        'customerId':{'field':{'name':'customer_id','kind':'string','description':'Google Ads customer.','default':''}},
        'body':{
          'query':'SELECT           campaign.name,           ad_group.name,           segments.geo_target_postal_code,           metrics.impressions,           metrics.clicks,           metrics.conversions,           metrics.interactions           FROM user_location_view         '
        }
      },
      'headers':{
        'developer-token':{'field':{'name':'developer_token','kind':'string','description':'Google Ads developer token.','default':''}},
        'login-customer-id':{'field':{'name':'login_id','kind':'string','description':'Google Ads login.','default':''}}
      },
      'iterate':True,
      'results':{
        'bigquery':{
          'dataset':{'field':{'name':'recipe_slug','kind':'string','suffix':'_Segmentology','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
          'table':'GoogleAds_KPI',
          'schema':[
            {
              'name':'userLocationView',
              'type':'RECORD',
              'mode':'NULLABLE',
              'fields':[
                {
                  'name':'resourceName',
                  'type':'STRING',
                  'mode':'NULLABLE'
                }
              ]
            },
            {
              'name':'segments',
              'type':'RECORD',
              'mode':'NULLABLE',
              'fields':[
                {
                  'name':'geoTargetPostalCode',
                  'type':'STRING',
                  'mode':'NULLABLE'
                }
              ]
            },
            {
              'name':'metrics',
              'type':'RECORD',
              'mode':'NULLABLE',
              'fields':[
                {
                  'name':'interactions',
                  'type':'INTEGER',
                  'mode':'NULLABLE'
                },
                {
                  'name':'impressions',
                  'type':'INTEGER',
                  'mode':'NULLABLE'
                },
                {
                  'name':'conversions',
                  'type':'INTEGER',
                  'mode':'NULLABLE'
                },
                {
                  'name':'clicks',
                  'type':'INTEGER',
                  'mode':'NULLABLE'
                }
              ]
            },
            {
              'name':'adGroup',
              'type':'RECORD',
              'mode':'NULLABLE',
              'fields':[
                {
                  'name':'name',
                  'type':'STRING',
                  'mode':'NULLABLE'
                },
                {
                  'name':'resourceName',
                  'type':'STRING',
                  'mode':'NULLABLE'
                }
              ]
            },
            {
              'name':'campaign',
              'type':'RECORD',
              'mode':'NULLABLE',
              'fields':[
                {
                  'name':'name',
                  'type':'STRING',
                  'mode':'NULLABLE'
                },
                {
                  'name':'resourceName',
                  'type':'STRING',
                  'mode':'NULLABLE'
                }
              ]
            }
          ]
        }
      }
    }
  },
  {
    'bigquery':{
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Authorization used for writing data.'}},
      'from':{
        'query':'SELECT            campaign.name AS Campaign,            adGRoup.name AS Ad_Group,            segments.geoTargetPostalCode AS Postal_Code,            SAFE_DIVIDE(metrics.impressions, SUM(metrics.impressions) OVER()) AS Impression,            SAFE_DIVIDE(metrics.clicks, metrics.impressions) AS Click,            SAFE_DIVIDE(metrics.conversions, metrics.impressions) AS Conversion,            SAFE_DIVIDE(metrics.interactions, metrics.impressions) AS Interaction,            metrics.impressions AS Impressions          FROM            `{dataset}.GoogleAds_KPI`;        ',
        'parameters':{
          'dataset':{'field':{'name':'recipe_slug','kind':'string','suffix':'_Segmentology','description':'Place where tables will be created in BigQuery.'}}
        },
        'legacy':False
      },
      'to':{
        'dataset':{'field':{'name':'recipe_slug','kind':'string','suffix':'_Segmentology','description':'Place where tables will be written in BigQuery.'}},
        'view':'GoogleAds_KPI_Normalized'
      }
    }
  },
  {
    'census':{
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Authorization used for writing data.'}},
      'normalize':{
        'census_geography':'zip_codes',
        'census_year':'2018',
        'census_span':'5yr'
      },
      'to':{
        'dataset':{'field':{'name':'recipe_slug','kind':'string','suffix':'_Segmentology','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
        'type':'view'
      }
    }
  },
  {
    'census':{
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Authorization used for writing data.'}},
      'correlate':{
        'join':'Postal_Code',
        'pass':[
          'Campaign',
          'Ad_Group'
        ],
        'sum':[
          'Impressions'
        ],
        'correlate':[
          'Impression',
          'Click',
          'Conversion',
          'Interaction'
        ],
        'dataset':{'field':{'name':'recipe_slug','kind':'string','suffix':'_Segmentology','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
        'table':'GoogleAds_KPI_Normalized',
        'significance':80
      },
      'to':{
        'dataset':{'field':{'name':'recipe_slug','kind':'string','suffix':'_Segmentology','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
        'type':'view'
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

execute(CONFIG, TASKS, force=True)
